In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pydicom

import os, sys
sys.path.append(os.path.abspath('..'))

from src.cine2gridtag import sim_gridtag, mod_contrast

In [4]:
from kedro.extras.datasets.pickle import PickleDataSet
from kedro.config import ConfigLoader

import os, sys
sys.path.append(os.path.abspath('..'))
sys.path.append(os.path.abspath('../src/'))

from src.tagseg.data.acdc_dataset import AcdcDataSet
from src.tagseg.data.dmd_dataset import DmdDataSet, DmdTimeDataSet
from src.tagseg.pipelines.data_processing.nodes import preprocess_acdc
from src.tagseg.pipelines.data_splitting.nodes import split_data

In [5]:
conf_paths = ["../conf/base", "../conf/local"]
conf_loader = ConfigLoader(conf_paths)
conf_catalog = conf_loader.get("catalog*", "catalog*/**")

In [7]:
dataset = DmdDataSet(filepath='../' + conf_catalog['dmd_data']['filepath']).load()

In [1]:
from monai.networks import nets

model = nets.SegResNetVAE(
        in_channels=1, out_channels=2,
        input_image_size=(256, 256), spatial_dims=2).double()

In [8]:
inp = dataset[0][0].unsqueeze(0).double()

out = model(inp)

In [11]:
out[0].shape

torch.Size([1, 2, 256, 256])

In [ ]:
# Load sample cine dicom
filepath = '../sample_data/im_s0035_0002.dcm'
ds = pydicom.read_file(filepath)

# This particular dataset is weirdly transposed, hence the [::-1] and .T
im = ds.pixel_array.astype(float).T[:, ::-1]

# This isn't really necessary, but helps window level consistency
im /= im.max()

In [ ]:
im_c = mod_contrast(im)

im_grid = sim_gridtag(im_c, 3.5)

fig, ax = plt.subplots(1, 3, figsize=(18, 6))

# fat is really bright in these images, right now I just manually set window level (clim)
ax[0].imshow(im, cmap='gray', clim=(0, 0.2)), ax[0].axis('off')
ax[1].imshow(im_c, cmap='gray', clim=(0, 0.7)), ax[1].axis('off')
ax[2].imshow(im_grid, cmap='gray', clim=(0, 0.7)), ax[2].axis('off')

In [ ]:
40/8